In [1]:
import time
import pandas as pd
#import modin.pandas as mpd
#import ray
#ignore_reinit_error=True
#ray.init()
#import polars as pl
#from datatable import dt, f, by, g, join, sort, update, ifelse

In [5]:
file_1 = '10_million_1.csv'
file_2 = '10_million_2.csv'
num_ran = 2

In [9]:
lib_name = 'pandas'
stats = []

for i in range (num_ran):
    stats_inner = {}
    start = time.time()
    df_1 = pd.read_csv(file_1,  engine="pyarrow", use_nullable_dtypes=True, index_col = 0)
    print(f'loading csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['loading_1'] = (time.time() - start)
    
    start = time.time()
    df_2 = pd.read_csv(file_2, engine="pyarrow", use_nullable_dtypes=True, index_col = 0)
    print(f'loading csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['loading_2'] = (time.time() - start)
    
    start = time.time()
    df_1.merge(df_2, on = 'unique_id_text' )
    print(f'merge csv on unique_id_text with {lib_name} took: {time.time() - start} seconds')
    stats_inner['merging'] = (time.time() - start)

    ## out of memory
    start=time.time()
    pd.concat([df_1, df_2])
    print(f'concat csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['concat'] = (time.time() - start)
    
    start=time.time()
    df_1.groupby('text_id').sum()
    print(f'groupby and sum by text_id with {lib_name} took: {time.time() - start} seconds ')
    stats_inner['groupby'] = (time.time() - start)
    
    start=time.time()
    df_1['new_col'] = df_1['0'].apply(round)
    print(f'apply round method with {lib_name} took {time.time() - start} seconds.')
    stats_inner['apply'] = (time.time() - start)
    
    start=time.time()
    df_1.loc[df_1['text_id'] == '9999_ID']
    print(f'.loc filter took {time.time() - start} seconds.')
    stats_inner['filtering'] = (time.time() - start)
    
    start=time.time()
    df_1.loc[df_1['text_id'] == '9999_ID', 'text_id'] = 'found it'
    print(f'.loc filter and update value took {time.time() - start} seconds.')
    stats_inner['filtering & updating'] = (time.time() - start)
    
    start=time.time()
    df_1['0_new'] = df_1['0'] * 2 + 1
    print(f'simple calculation took {time.time() - start} seconds.')
    stats_inner['column calculation'] = (time.time() - start)
    
    stats.append(stats_inner)
    
print([k for k in sorted(stats[0].keys())])
pandas_stats = [round((stats[0][k] + stats[1][k])/2,5) for k in sorted(stats[0].keys())]
print(pandas_stats)

TypeError: read_csv() got an unexpected keyword argument 'use_nullable_dtypes'

In [11]:
df_1 = pd.read_csv(file_1,  engine="pyarrow", index_col = 0)

In [17]:
df_1.head()

,0,1,2,3,4,5,6,integer_id,text_id,unique_id_text
,,,,,,,,,,
8853495,0.855285,0.240642,0.732951,0.860857,0.956267,0.416427,0.050623,4780,4780_ID,8853495_ID
4844597,0.901882,0.999457,0.978952,0.559693,0.859066,0.737484,0.713445,8340,8340_ID,4844597_ID
3912746,0.350191,0.766841,0.572385,0.496385,0.503123,0.646241,0.951089,4882,4882_ID,3912746_ID
3143505,0.311571,0.042855,0.774996,0.232767,0.301296,0.576442,0.217577,9588,9588_ID,3143505_ID
1030606,0.848233,0.081258,0.754872,0.809918,0.961630,0.065941,0.284760,8066,8066_ID,1030606_ID


In [19]:
df_1['unique_id_text']


8853495    8853495_ID
4844597    4844597_ID
3912746    3912746_ID
3143505    3143505_ID
1030606    1030606_ID
              ...    
402204      402204_ID
5829411    5829411_ID
1434403    1434403_ID
6754697    6754697_ID
5921685    5921685_ID
Name: unique_id_text, Length: 10000000, dtype: object

# modin

In [3]:
lib_name = 'modin'

stats = []

for i in range (num_ran):
    stats_inner = {}
    start = time.time()
    df_1 = mpd.read_csv(file_1,  index_col = 0)
    print(f'loading csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['loading_1'] = (time.time() - start)
    
    start = time.time()
    df_2 = mpd.read_csv(file_2,  index_col = 0)
    print(f'loading csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['loading_2'] = (time.time() - start)
    
#     start = time.time()
#     df_1.merge(df_2, on = 'unique_id_text' )
#     print(f'merge csv on unique_id_text with {lib_name} took: {time.time() - start} seconds')
#     stats_inner['merging'] = (time.time() - start)

    ## out of memory
    start=time.time()
    mpd.concat([df_1, df_2])
    print(f'concat csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['concat'] = (time.time() - start)
    
    start=time.time()
    df_1.groupby('text_id').sum()
    print(f'groupby and sum by text_id with {lib_name} took: {time.time() - start} seconds ')
    stats_inner['groupby'] = (time.time() - start)
    
    start=time.time()
    df_1['new_col'] = df_1['0'].apply(round)
    print(f'apply round method with {lib_name} took {time.time() - start} seconds.')
    stats_inner['apply'] = (time.time() - start)
    
    start=time.time()
    df_1.loc[df_1['text_id'] == '9999_ID']
    print(f'.loc filter took {time.time() - start} seconds.')
    stats_inner['filtering'] = (time.time() - start)
    
    start=time.time()
    df_1.loc[df_1['text_id'] == '9999_ID', 'text_id'] = 'found it'
    print(f'.loc filter and update value took {time.time() - start} seconds.')
    stats_inner['filtering & updating'] = (time.time() - start)
    
    start=time.time()
    df_1['0_new'] = df_1['0'] * 2 + 1
    print(f'simple calculation took {time.time() - start} seconds.')
    stats_inner['column calculation'] = (time.time() - start)
    
    stats.append(stats_inner)
    
print([k for k in sorted(stats[0].keys())])
modin_stats = [round((stats[0][k] + stats[1][k])/2,5) for k in sorted(stats[0].keys())]
print(modin_stats)

loading csv with modin took: 1.4963836669921875 seconds
loading csv with modin took: 0.46642446517944336 seconds
concat csv with modin took: 0.005003929138183594 seconds
groupby and sum by text_id with modin took: 1.2561426162719727 seconds 
apply round method with modin took 0.08707928657531738 seconds.
.loc filter took 0.09308457374572754 seconds.
.loc filter and update value took 0.08808016777038574 seconds.
simple calculation took 0.07406759262084961 seconds.
loading csv with modin took: 0.7657451629638672 seconds
loading csv with modin took: 0.38288092613220215 seconds
concat csv with modin took: 0.0030024051666259766 seconds
groupby and sum by text_id with modin took: 1.5353965759277344 seconds 
apply round method with modin took 0.08707928657531738 seconds.
.loc filter took 0.09108328819274902 seconds.
.loc filter and update value took 0.09008121490478516 seconds.
simple calculation took 0.3478207588195801 seconds.
['apply', 'column calculation', 'concat', 'filtering', 'filterin

# polars

In [ ]:
lib_name = 'polars'
stats = []

for i in range(num_ran):
    stats_inner = {}
    start = time.time()
    df_1 = pl.read_csv(file_1)
    print(f'loading csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['loading_1'] = (time.time() - start)

    start = time.time()
    df_2 = pl.read_csv(file_2)
    print(f'loading csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['loading_2'] = (time.time() - start)

    start=time.time()
    pl.concat([df_1, df_2])
    print(f'concat (append) csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['concat'] = (time.time() - start)

    start=time.time()
    df_1.join(df_2, on = 'unique_id_text')
    print(f'join (merge) csv with {lib_name} took: {time.time() - start} seconds')
    stats_inner['merging'] = (time.time() - start)

    start=time.time()
    df_1.groupby('text_id').sum()
    print(f'groupby and sum by text_id with {lib_name} took: {time.time() - start} seconds ')
    stats_inner['groupby'] = (time.time() - start)

    start=time.time()
    df_1['new_col'] = df_1['0'].apply(round)
    print(f'apply round method with {lib_name} took {time.time() - start} seconds.')
    stats_inner['apply'] = (time.time() - start)

    start=time.time()
    df_1[df_1['text_id'] == '9999_ID']
    print(f'filter took {time.time() - start} seconds.')
    stats_inner['filtering'] = (time.time() - start)

    start=time.time()
    df_1[df_1['text_id'] == '9999_ID', 'text_id'] = 'found it'
    print(f'filter and update value took {time.time() - start} seconds.')
    stats_inner['filtering & updating'] = (time.time() - start)

    start=time.time()
    df_1['0_2'] = df_1['0'] *2 + 1
    print(f'simple column calculation with {lib_name} took {time.time() - start} seconds.')
    stats_inner['column calculation'] = (time.time() - start)
    
    stats.append(stats_inner)

print([k for k in sorted(stats[0].keys())])
polars_stats = [round((stats[0][k] + stats[1][k])/2,5) for k in sorted(stats[0].keys())]
print(polars_stats)

In [ ]:
df_1 = pl.read_csv(file_1)

In [ ]:
df_1.head()

# datatable

In [ ]:
lib_name = 'datatable'
stats = []

for i in range(num_ran):
    stats_inner = {}
    start = time.time()
    df_1 = dt.fread(file_1)
    print(f'loading csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['loading_1'] = (time.time() - start)
    
    start = time.time()
    df_2 = dt.fread(file_2)
    print(f'loading csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['loading_2'] = (time.time() - start)

    start = time.time()
    df_2 = dt.rbind(df_1, df_2)
    print(f'vertically concat csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['concat'] = (time.time() - start)
    
    df_3 = df_1[:,0:10]
    start = time.time()
    df_3[:,dt.sum(f[:]), by('text_id')]
    print(f'groupby and sum csv with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['groupby'] = (time.time() - start)
    
    start = time.time()
    df_1['0_2'] = f[1] * 2 +1
    print(f'simple column calculation with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['column calculation'] = (time.time() - start)
    
    start = time.time()
    df_1[f.text_id == '9999_ID',:]
    print(f'filtering with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['filtering'] = (time.time() - start)
    
    start = time.time()
    df_1[f.text_id == '9999_ID','text_id'] = 'found it'
    print(f'filtering and update value with {lib_name} took: {time.time()-start} seconds.')
    stats_inner['filtering & updating'] = (time.time() - start)
    
    stats_inner['merge'] = 0
    stats.append(stats_inner)

print([k for k in sorted(stats[0].keys())])
datatable_stats = [round((stats[0][k] + stats[1][k])/2,5) for k in sorted(stats[0].keys())]
print(datatable_stats)

In [ ]:
df_1.head()

In [ ]:
df_1

In [ ]:
%time test('pandas')

In [ ]:
%time test('modin')

In [ ]:
import polars as pl

In [ ]:
df1 = pl.read_csv(file_1)
df2 = pl.read_csv(file_2)
df3 = pl.concat([df1, df2])

In [ ]:
df3.shape

In [ ]:
df4 = df1.join(df2, on ='unique_id_text')

In [ ]:
df4.shape

In [ ]:
df4.head()

In [ ]:
df_1.groupby('text_id').sum()